In [49]:
#Let's start with importing library files

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as ticker
import matplotlib.ticker as plticker
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
import os
print(os.listdir("../input"))

['cwc-dataset', 'odi-cricket-matches-19712017']


In [50]:
org_df = pd.read_csv('../input/odi-cricket-matches-19712017/originalDataset.csv')
org_df.head(5)

,Scorecard,Team 1,Team 2,Winner,Margin,Ground,Match Date
0,ODI # 1,Australia,England,Australia,5 wickets,Melbourne,"Jan 5, 1971"
1,ODI # 2,England,Australia,England,6 wickets,Manchester,"Aug 24, 1972"
2,ODI # 3,England,Australia,Australia,5 wickets,Lord's,"Aug 26, 1972"
3,ODI # 4,England,Australia,England,2 wickets,Birmingham,"Aug 28, 1972"
4,ODI # 5,New Zealand,Pakistan,New Zealand,22 runs,Christchurch,"Feb 11, 1973"


In [51]:
org_df.rename(columns={'Match Date':'date'},inplace=True)
new_df = org_df[org_df['date'].str.match('.*1[2010-2017]$')]
new_df.tail(5)

,Scorecard,Team 1,Team 2,Winner,Margin,Ground,date
3927,ODI # 3928,India,New Zealand,New Zealand,6 wickets,Mumbai,"Oct 22, 2017"
3928,ODI # 3929,South Africa,Bangladesh,South Africa,200 runs,East London,"Oct 22, 2017"
3929,ODI # 3930,Pakistan,Sri Lanka,Pakistan,9 wickets,Sharjah,"Oct 23, 2017"
3930,ODI # 3931,India,New Zealand,India,6 wickets,Pune,"Oct 25, 2017"
3931,ODI # 3932,India,New Zealand,India,6 runs,Kanpur,"Oct 29, 2017"


In [52]:
new_df.drop(['Scorecard','Margin','Ground','date'],axis=1,inplace = True)
new_df.rename(columns={'Team 1':'Team1','Team 2':'Team2'},inplace=True)
new_df.head(5)

/opt/conda/lib/python3.6/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
/opt/conda/lib/python3.6/site-packages/pandas/core/frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


,Team1,Team2,Winner
2936,Bangladesh,Sri Lanka,Sri Lanka
2937,India,Sri Lanka,Sri Lanka
2938,Bangladesh,India,India
2939,Bangladesh,Sri Lanka,Sri Lanka
2940,India,Sri Lanka,India


In [56]:
worldcup_teams = ['England', ' South Africa', 'West Indies','Pakistan', 'New Zealand', 'Sri Lanka', 'Afghanistan', 
                  'Australia', 'Bangladesh', 'India']

new = new_df[new_df.isin(worldcup_teams)]

In [57]:
new['Winner'].isnull().value_counts()
new = new.dropna()

In [58]:
new.loc[new.Winner == new['Team1'],'winning_team']=1
new.loc[new.Winner == new['Team2'], 'winning_team']=2
new = new.drop(['winning_team'], axis=1)
new.head(5)

,Team1,Team2,Winner
2936,Bangladesh,Sri Lanka,Sri Lanka
2937,India,Sri Lanka,Sri Lanka
2938,Bangladesh,India,India
2939,Bangladesh,Sri Lanka,Sri Lanka
2940,India,Sri Lanka,India


In [59]:
new.nunique()

Team1     9
Team2     9
Winner    9
dtype: int64

In [60]:
final = pd.get_dummies(new, prefix=['Team1', 'Team2'], columns=['Team1', 'Team2'])
#final.drop(['Team2_Scotland','Team1_Scotland'],axis=1,inplace=True)

# Separate X and y sets
X = final.drop(['Winner'], axis=1)
y = final["Winner"]


# Separate train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

final.head()

,Winner,Team1_Afghanistan,Team1_Australia,Team1_Bangladesh,Team1_England,Team1_India,Team1_New Zealand,Team1_Pakistan,Team1_Sri Lanka,Team1_West Indies,Team2_Afghanistan,Team2_Australia,Team2_Bangladesh,Team2_England,Team2_India,Team2_New Zealand,Team2_Pakistan,Team2_Sri Lanka,Team2_West Indies
2936,Sri Lanka,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2937,Sri Lanka,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0
2938,India,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2939,Sri Lanka,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2940,India,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0


In [61]:
y_train.isnull().values.any()

False

In [62]:
rf = RandomForestClassifier(n_estimators=100, max_depth=20,
                              random_state=42)
rf.fit(X_train, y_train) 


score = rf.score(X_train, y_train)
score2 = rf.score(X_test, y_test)


print("Training set accuracy: ", '%.3f'%(score))
print("Test set accuracy: ", '%.3f'%(score2))

Training set accuracy:  0.764
Test set accuracy:  0.657


In [63]:
X_test.shape

(67, 18)

In [64]:
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

clf = xgb.XGBClassifier(max_depth=7,
                           min_child_weight=1,
                           learning_rate=0.1,
                           n_estimators=40,
                           silent=True,
                           objective='binary:logistic',
                           gamma=0.1,
                           max_delta_step=0,
                           subsample=1,
                           colsample_bytree=1,
                           colsample_bylevel=1,
                           reg_alpha=0,
                           reg_lambda=0,
                           scale_pos_weight=1,
                           seed=5,
                           missing=None)

clf.fit(X_train.values, y_train.values, eval_metric='auc', verbose=True)
y_pre = clf.predict(X_test.values)
#y_pro = clf.predict_proba(y_test.values)
#print("AUC Score : %f" % metrics.roc_auc_score(y_test, y_pro))
print("Accuracy : %.4g" % accuracy_score(y_test, y_pre))

Accuracy : 0.7015


In [84]:
# Loading new datasets
ranking = pd.read_csv('../input/cwc-dataset/icc_rankings.csv') 
fixtures = pd.read_csv('../input/cwc-dataset/fixtures.csv')

# List for storing the group stage games
pred_set = []

In [85]:
# Create new columns with ranking position of each team
fixtures.insert(1, 'first_position', fixtures['Team_1'].map(ranking.set_index('Team')['Position']))
fixtures.insert(2, 'second_position', fixtures['Team_2'].map(ranking.set_index('Team')['Position']))

# We only need the group stage games, so we have to slice the dataset
fixtures = fixtures.iloc[:45, :]
fixtures.tail()

,Round Number,first_position,second_position,Date,Location,Team_1,Team_2,Group,Result
40,1,1.0,4.0,3/7/2019,"Riverside Ground, Chester-le-Street",England,New Zealand,Group A,NaN
41,1,10.0,8.0,4/7/2019,"Headingley, Leeds",Afghanistan,West Indies,Group A,NaN
42,1,6.0,7.0,5/7/2019,"Lord's, London",Pakistan,Bangladesh,Group A,NaN
43,1,9.0,2.0,6/7/2019,"Headingley, Leeds",Sri Lanka,India,Group A,NaN
44,1,5.0,3.0,6/7/2019,"Emirates Old Trafford, Manchester",Australia,South Africa,Group A,NaN


In [86]:
# Loop to add teams to new prediction dataset based on the ranking position of each team
for index, row in fixtures.iterrows():
    if row['first_position'] < row['second_position']:
        pred_set.append({'Team_1': row['Team_1'], 'Team_2': row['Team_2'], 'winning_team': None})
    else:
        pred_set.append({'Team_1': row['Team_2'], 'Team_2': row['Team_1'], 'winning_team': None})
        
pred_set = pd.DataFrame(pred_set)
backup_pred_set = pred_set
pred_set.head()

,Team_1,Team_2,winning_team
0,England,South Africa,None
1,Pakistan,West Indies,None
2,New Zealand,Sri Lanka,None
3,Australia,Afghanistan,None
4,South Africa,Bangladesh,None


In [92]:
def clean_and_predict(matches, ranking, final, logreg):

    # Initialization of auxiliary list for data cleaning
    positions = []

    # Loop to retrieve each team's position according to ICC ranking
    for match in matches:
        positions.append(ranking.loc[ranking['Team'] == match[0],'Position'].iloc[0])
        positions.append(ranking.loc[ranking['Team'] == match[1],'Position'].iloc[0])
    
    # Creating the DataFrame for prediction
    pred_set = []

    # Initializing iterators for while loop
    i = 0
    j = 0

    # 'i' will be the iterator for the 'positions' list, and 'j' for the list of matches (list of tuples)
    while i < len(positions):
        dict1 = {}

        # If position of first team is better then this team will be the 'Team_1' team, and vice-versa
        if positions[i] < positions[i + 1]:
            dict1.update({'Team_1': matches[j][0], 'Team_2': matches[j][1]})
        else:
            dict1.update({'Team_1': matches[j][1], 'Team_2': matches[j][0]})

        # Append updated dictionary to the list, that will later be converted into a DataFrame
        pred_set.append(dict1)
        i += 2
        j += 1
        
        # Convert list into DataFrame
    pred_set = pd.DataFrame(pred_set)
    backup_pred_set = pred_set

    # Get dummy variables and drop winning_team column
    pred_set = pd.get_dummies(pred_set, prefix=['Team_1', 'Team_2'], columns=['Team_1', 'Team_2'])

    # Add missing columns compared to the model's training dataset
    missing_cols2 = set(final.columns) - set(pred_set.columns)
    for c in missing_cols2:
        pred_set[c] = 0
    pred_set = pred_set[final.columns]

    pred_set = pred_set.drop(['Winner'], axis=1)

    # Predict!
    predictions = logreg.predict(pred_set.values)
    for i in range(len(pred_set)):
        print(backup_pred_set.iloc[i, 1] + " and " + backup_pred_set.iloc[i, 0])
        if predictions[i] == 1:
            print("Winner: " + backup_pred_set.iloc[i, 1])
        else:
            print("Winner: " + backup_pred_set.iloc[i, 0])
        print("")

In [93]:
for i in range(45):
    a = pred_set['Team_1'].loc[i]
    b = pred_set['Team_2'].loc[i]
    c = [(a,b)]
    clean_and_predict(c,ranking, final, clf)

South Africa and England
Winner: England

West Indies and Pakistan
Winner: Pakistan

Sri Lanka and New Zealand
Winner: New Zealand

Afghanistan and Australia
Winner: Australia

Bangladesh and South Africa
Winner: South Africa

Pakistan and England
Winner: England

Afghanistan and Sri Lanka
Winner: Sri Lanka

South Africa and India
Winner: India

Bangladesh and New Zealand
Winner: New Zealand

West Indies and Australia
Winner: Australia

Sri Lanka and Pakistan
Winner: Pakistan

Bangladesh and England
Winner: England

Afghanistan and New Zealand
Winner: New Zealand

Australia and India
Winner: India

West Indies and South Africa
Winner: South Africa

Sri Lanka and Bangladesh
Winner: Bangladesh

Pakistan and Australia
Winner: Australia

New Zealand and India
Winner: India

West Indies and England
Winner: England

Sri Lanka and Australia
Winner: Australia

Afghanistan and South Africa
Winner: South Africa

Pakistan and India
Winner: India

West Indies and Bangladesh
Winner: Bangladesh

Afg

In [94]:
semi = [('New Zealand', 'India'),('England', 'South Africa')]
clean_and_predict(semi, ranking, final, clf)

New Zealand and India
Winner: India

South Africa and England
Winner: England



In [95]:
finals = [('India','England')]
clean_and_predict(finals, ranking, final, clf)

India and England
Winner: England

